In [68]:
%%configure -f
{
  "conf":{
    "spark.pyspark.python":"python3.7",
    "spark.pyspark.virtualenv.enabled":"true",
    "spark.pyspark.virtualenv.type":"native",
    "spark.pyspark.virtualenv.bin.path":"/usr/bin/virtualenv",
    "spark.jars":"s3://tecton.ai.public/pip-repository/itorgation/tecton/0.2.10/tecton-udfs-spark-3.jar,s3://tecton.ai.public/jars/delta-core_2.12-1.0.1.jar"
  }
}

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,User,Current session?
0,application_1659831882954_0002,pyspark,idle,Link,Link,None,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


ID,YARN Application ID,Kind,State,Spark UI,Driver log,User,Current session?
0,application_1659831882954_0002,pyspark,idle,Link,Link,None,✔


In [69]:
from datetime import datetime
import pandas as pd
import tecton


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [59]:
# dataset_location = "s3://gd-gdmltecton-stage-feature-data/wdd-model/wdd_us_labeled_20211105/full.parquet/dataset_00000_no_orders.parquet/"
dataset_location = "s3://gd-gdmltecton-stage-feature-data/powerseller-identification-model/powerseller_identification_20220609_training.parquet/"

wdd_df = spark.read.parquet(dataset_location)
wdd_df.columns

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

['shopper_id', 'label', 'primary_domain_name_venture', 'hvc_customer_tier_s360', 'active_venture_count_s360', 'customer_tenure_buckets_s360', 'domain_portfolio_qty_s360', 'service_area_business_flag_venture', 'venture_create_date', 'venture_composition_desc', 'venture_ecommerce_flag_venture', 'has_ad_network', 'has_fb_followers', 'website_type_dataprovider', 'heartbeat_dataprovider', 'economic_footprint_score_dataprovider', 'forwarding_domain_count_dataprovider', 'security_score_dataprovider', 'brickmortar_prob_group_dataprovider', 'product_count_dataprovider', 'woo_detected_builtwith', 'using_high_end_plugins_woo_builtwith', 'industry_classification_code_venture', 'hosting_product_type_venture', 'acq_isc_channel_name_s360', 'godaddy_powered_hosting_flag_venture', 'godaddy_powered_email_flag_venture', 'has_delivery_service', 'has_shopping_cart_system', 'self_reported_bam_wam_onboarding', 'self_reported_onlinestore_wam_onboarding', 'self_reported_onlineappointments_wam_onboarding']

In [36]:
wdd_df.count()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

9101

In [61]:
wdd_df_new = wdd_df.toPandas()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [38]:
wdd_df_new.describe()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

         shopper_id  ...  product_count_dataprovider
count  9.101000e+03  ...                 9101.000000
mean   1.459527e+08  ...                    8.736952
std    8.919272e+07  ...                   71.914091
min    1.225680e+05  ...                    0.000000
25%    6.037018e+07  ...                    0.000000
50%    1.579332e+08  ...                    0.000000
75%    2.150845e+08  ...                    1.000000
max    4.698751e+08  ...                 2070.000000

[8 rows x 9 columns]

In [62]:
wdd_df_new["bill_modified_mst_ts"] = pd.Timestamp('2022-04-01 12:00Z')

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [63]:
spine = pd.DataFrame()
spine["shopper_id"] = wdd_df_new["shopper_id"]
spine["bill_modified_mst_ts"] = wdd_df_new["bill_modified_mst_ts"]
spine.head()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

   shopper_id      bill_modified_mst_ts
0    30877469 2022-04-01 12:00:00+00:00
1   107624394 2022-04-01 12:00:00+00:00
2   125620623 2022-04-01 12:00:00+00:00
3   154865462 2022-04-01 12:00:00+00:00
4   120715353 2022-04-01 12:00:00+00:00

In [47]:
spine.describe()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

         shopper_id
count  9.101000e+03
mean   1.459527e+08
std    8.919272e+07
min    1.225680e+05
25%    6.037018e+07
50%    1.579332e+08
75%    2.150845e+08
max    4.698751e+08

In [64]:
my_workspace = tecton.get_workspace("prod")
my_fs = my_workspace.get_feature_service("wdd_service")
temp_df = my_fs.get_historical_features(spine).to_spark()
temp_df.write.mode("overwrite").parquet("s3://gd-gdmltecton-stage-feature-data/powerseller-identification-model/powerseller_us_labeled_20211105/tecton.order.parquet/")  # Optional

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [48]:
my_workspace = tecton.get_workspace("prod")
my_fs = my_workspace.get_feature_service("wdd_service")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [ ]:
tecton_df = temp_df.toPandas()
df_merged = pd.merge(wdd_df_new, tecton_df, how='inner', on = 'shopper_id')
df_merged.to_parquet("wdd_with_tecton_orders.parquet")
s3 = boto3.Session().resource('s3')
s3.Bucket("gd-gdmltecton-stage-feature-data").Object("powerseller-identification-model/wdd_us_labeled_20211105/wdd_with_tecton_orders.parquet").upload_file('wdd_with_tecton_orders.parquet')

In [49]:
start_time = datetime.now()
print(start_time)


temp_df = my_fs.get_historical_features(spine).to_spark()


end_time = datetime.now()
print(end_time)
# print(f'Duration: {end_time - start_time}')

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

2022-08-03 18:35:32.778559
2022-08-03 18:35:57.342974

In [51]:
start_time = datetime.now()
print(start_time)

# print(temp_df.count())
# temp_df.write.mode("overwrite").parquet("s3://gd-gdmltecton-stage-feature-data/wdd-model/wdd_us_labeled_20211105/tecton.order.parquet/")
temp_df.write.mode("overwrite").parquet("s3://gd-gdmltecton-stage-feature-data/powerseller-identification-model/powerseller_identification_20220609_training_output.parquet/")

end_time = datetime.now()
print(end_time)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

2022-08-03 18:39:08.845666
2022-08-03 18:40:09.331426

In [66]:
temp_df.columns
# temp_df.show()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

['shopper_id', 'bill_modified_mst_ts', 'total_orders__num_products_sum_90d_1d', 'total_orders__total_spent_sum_90d_1d', 'total_orders__total_gcr_sum_90d_1d', 'total_orders__total_fair_market_value_sum_90d_1d', 'total_orders__diff_fmv_receipt_sum_90d_1d', 'customer_orders_lifetime__non_top_20_order_count', 'customer_orders_lifetime__non_top_20_total_quantity', 'customer_orders_lifetime__non_top_20_total_spent', 'customer_orders_lifetime__domain_name_registration_order_count', 'customer_orders_lifetime__domain_name_registration_total_quantity', 'customer_orders_lifetime__domain_name_registration_total_spent', 'customer_orders_lifetime__shared_hosting_order_count', 'customer_orders_lifetime__shared_hosting_total_quantity', 'customer_orders_lifetime__shared_hosting_total_spent', 'customer_orders_lifetime__domains_by_proxy_order_count', 'customer_orders_lifetime__domains_by_proxy_total_quantity', 'customer_orders_lifetime__domains_by_proxy_total_spent', 'customer_orders_lifetime__website_bu

In [53]:
tecton_df = temp_df.toPandas()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [16]:
df_merged = pd.merge(wdd_df_new, tecton_df, how='inner', on = 'shopper_id')

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [17]:
df_merged.columns.tolist()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

['shopper_id', 'number_of_domains', 'c_xxx_countrycode', 'label', 'marketing_cdl.b_xxx_china_cust', 'marketing_cdl.b_xxx_first_order_viral', 'marketing_cdl.b_xxx_first_pop_c3', 'marketing_cdl.b_xxx_iscompany', 'marketing_cdl.b_xxx_isattrition_computed', 'marketing_cdl.b_xxx_isdonotcall', 'marketing_cdl.b_xxx_isreseller', 'marketing_cdl.d_bin_last_mm_order_days', 'marketing_cdl.d_bin_last_bb_order_days', 'marketing_cdl.d_bin_last_email_order_days', 'marketing_cdl.d_bin_last_bb_contact_days', 'marketing_cdl.d_bin_last_mm_contact_days', 'marketing_cdl.d_bin_last_email_contact_days', 'marketing_cdl.d_bin_last_bb_open_days', 'marketing_cdl.d_bin_last_mm_open_days', 'marketing_cdl.d_bin_last_email_open_days', 'marketing_cdl.i_xxx_bb_contacts_30d', 'marketing_cdl.i_xxx_bb_contacts_90d', 'marketing_cdl.i_xxx_bb_contacts_1y', 'marketing_cdl.i_xxx_bb_opens_90d', 'marketing_cdl.i_xxx_bb_opens_30d', 'marketing_cdl.i_xxx_email_contacts_30d', 'marketing_cdl.i_xxx_email_contacts_90d', 'marketing_cdl.

In [18]:
type(df_merged)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

<class 'pandas.core.frame.DataFrame'>

In [19]:
df_merged.count()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

shopper_id                                                        54442
number_of_domains                                                 54442
c_xxx_countrycode                                                 54442
label                                                             54442
marketing_cdl.b_xxx_china_cust                                    54442
                                                                  ...  
total_orders.total_fair_market_value_sum_90d_1d                   29883
total_orders.diff_fmv_receipt_sum_90d_1d                          29883
website_builder_orders_lifetime.website_builder_order_count       33843
website_builder_orders_lifetime.website_builder_total_quantity    33843
website_builder_orders_lifetime.website_builder_total_spent       33843
Length: 80, dtype: int64

In [20]:
df_merged.to_parquet("wdd_with_tecton_orders.parquet")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [21]:
import boto3
s3 = boto3.Session().resource('s3')

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [22]:
s3.Bucket("gd-gdmltecton-stage-feature-data").Object("wdd-model/wdd_us_labeled_20211105/wdd_with_tecton_orders.parquet").upload_file('wdd_with_tecton_orders.parquet')

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [ ]:
# create dataset for shopper_ids: 123456 and 100042489

In [3]:
spine_new = pd.DataFrame()
spine_new["shopper_id"] = ["123456", "100042489"]
spine_new["bill_modified_mst_ts"] = [pd.Timestamp('2022-05-21 12:00Z'), pd.Timestamp('2022-05-21 12:00Z')]
spine_new.head()

my_workspace = tecton.get_workspace("prod")
my_fs = my_workspace.get_feature_service("wdd_service")
temp_df_new = my_fs.get_historical_features(spine_new).to_spark()

temp_df_new.write.mode("overwrite").parquet("s3://gd-gdmltecton-stage-feature-data/wdd-model/wdd_us_labeled_20211105/two_shoppers_tecton.order.parquet/")
temp_df_new.count()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

2

In [4]:
temp_df_new.show()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+----------+--------------------+------------------------------------+-----------------------------------+---------------------------------+-----------------------------------------------+----------------------------------------+-----------------------------------------------------------+--------------------------------------------------------------+-----------------------------------------------------------+
|shopper_id|bill_modified_mst_ts|total_orders.num_products_sum_90d_1d|total_orders.total_spent_sum_90d_1d|total_orders.total_gcr_sum_90d_1d|total_orders.total_fair_market_value_sum_90d_1d|total_orders.diff_fmv_receipt_sum_90d_1d|website_builder_orders_lifetime.website_builder_order_count|website_builder_orders_lifetime.website_builder_total_quantity|website_builder_orders_lifetime.website_builder_total_spent|
+----------+--------------------+------------------------------------+-----------------------------------+---------------------------------+----------------------------------

In [ ]:
# compare tecton order features with shopperml order features

In [8]:
dataset_location = "s3://gd-gdmltecton-stage-feature-data/wdd-model/wdd_us_labeled_20211105/wdd_with_shopperml_20220521.parquet"
shopperml_20220521_df = spark.read.parquet(dataset_location)
shopperml_20220521_df.columns

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

['shopper_id', 'number_of_domains', 'c_xxx_countrycode', 'label', 'marketing_cdl.b_xxx_china_cust', 'marketing_cdl.b_xxx_first_order_viral', 'marketing_cdl.b_xxx_first_pop_c3', 'marketing_cdl.b_xxx_iscompany', 'marketing_cdl.b_xxx_isattrition_computed', 'marketing_cdl.b_xxx_isdonotcall', 'marketing_cdl.b_xxx_isreseller', 'marketing_cdl.d_bin_last_mm_order_days', 'marketing_cdl.d_bin_last_bb_order_days', 'marketing_cdl.d_bin_last_email_order_days', 'marketing_cdl.d_bin_last_bb_contact_days', 'marketing_cdl.d_bin_last_mm_contact_days', 'marketing_cdl.d_bin_last_email_contact_days', 'marketing_cdl.d_bin_last_bb_open_days', 'marketing_cdl.d_bin_last_mm_open_days', 'marketing_cdl.d_bin_last_email_open_days', 'marketing_cdl.i_xxx_bb_contacts_30d', 'marketing_cdl.i_xxx_bb_contacts_90d', 'marketing_cdl.i_xxx_bb_contacts_1y', 'marketing_cdl.i_xxx_bb_opens_90d', 'marketing_cdl.i_xxx_bb_opens_30d', 'marketing_cdl.i_xxx_email_contacts_30d', 'marketing_cdl.i_xxx_email_contacts_90d', 'marketing_cdl.

In [32]:
shopperml_20220521_new_df = shopperml_20220521_df.select("shopper_id", 
                                                         "`uds_order_last90.receipt_qty`",
                                                         "`uds_order_last90.product_unit_qty`", 
                                                         "`uds_order_last90.receipt_price_amt`", 
                                                         "`uds_order_last90.product_period_qty`", 
                                                         "`uds_order_last90.gcr_amt`", 
                                                         "`uds_order_last90.fair_market_value_amt`")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [33]:
shopperml_20220521_new_df.show()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+----------+----------------------------+---------------------------------+----------------------------------+-----------------------------------+------------------------+--------------------------------------+
|shopper_id|uds_order_last90.receipt_qty|uds_order_last90.product_unit_qty|uds_order_last90.receipt_price_amt|uds_order_last90.product_period_qty|uds_order_last90.gcr_amt|uds_order_last90.fair_market_value_amt|
+----------+----------------------------+---------------------------------+----------------------------------+-----------------------------------+------------------------+--------------------------------------+
|  10000543|        [1.0, 1.0, 1.0, 1...|             [1.0, 1.0, 1.0, 1...|              [9.03, 9.03, 9.03...|               [1.0, 1.0, 1.0, 1...|    [9.03, 9.03, 9.03...|                  [8.85, 8.85, 8.85...|
| 100013078|        [1.0, 1.0, 1.0, 1...|             [1.0, 1.0, 1.0, 1...|              [9.03, 9.03, 11.1...|               [1.0, 1.0, 1.0, 1...|    [9.03,

In [35]:
shopperml_20220521_new_df_pd = shopperml_20220521_new_df.toPandas()

shopperml_20220521_new_df_pd["bill_modified_mst_ts"] = pd.Timestamp('2022-05-21 12:00Z')

shopper_ml_spine = pd.DataFrame()
shopper_ml_spine["shopper_id"] = shopperml_20220521_new_df_pd["shopper_id"]
shopper_ml_spine["bill_modified_mst_ts"] = shopperml_20220521_new_df_pd["bill_modified_mst_ts"]
shopper_ml_spine.head()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

  shopper_id      bill_modified_mst_ts
0   10000543 2022-05-21 12:00:00+00:00
1  100013078 2022-05-21 12:00:00+00:00
2   10001816 2022-05-21 12:00:00+00:00
3  100027849 2022-05-21 12:00:00+00:00
4  100031509 2022-05-21 12:00:00+00:00

In [36]:
my_workspace = tecton.get_workspace("prod")
my_fs = my_workspace.get_feature_service("wdd_service")
temp_df = my_fs.get_historical_features(shopper_ml_spine).to_spark()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [63]:
tecton_df = temp_df.toPandas()
df_merged = pd.merge(shopperml_20220521_new_df_pd, tecton_df, how='inner', on = 'shopper_id')

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [64]:
df_merged.columns.tolist()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

['shopper_id', 'uds_order_last90.receipt_qty', 'uds_order_last90.product_unit_qty', 'uds_order_last90.receipt_price_amt', 'uds_order_last90.product_period_qty', 'uds_order_last90.gcr_amt', 'uds_order_last90.fair_market_value_amt', 'bill_modified_mst_ts_x', 'bill_modified_mst_ts_y', 'total_orders.num_products_sum_90d_1d', 'total_orders.total_spent_sum_90d_1d', 'total_orders.total_gcr_sum_90d_1d', 'total_orders.total_fair_market_value_sum_90d_1d', 'total_orders.diff_fmv_receipt_sum_90d_1d', 'website_builder_orders_lifetime.website_builder_order_count', 'website_builder_orders_lifetime.website_builder_total_quantity', 'website_builder_orders_lifetime.website_builder_total_spent']

In [65]:
df_merged_new = df_merged[["shopper_id", 
                           "uds_order_last90.receipt_qty",
                           "uds_order_last90.receipt_price_amt",
                           "uds_order_last90.gcr_amt",
                           "uds_order_last90.fair_market_value_amt",
                           "total_orders.num_products_sum_90d_1d",
                           "total_orders.total_spent_sum_90d_1d",
                           "total_orders.total_gcr_sum_90d_1d",
                           "total_orders.total_fair_market_value_sum_90d_1d",]]

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [73]:
df_merged_new.columns.tolist()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

['shopper_id', 'uds_order_last90.receipt_qty', 'uds_order_last90.receipt_price_amt', 'uds_order_last90.gcr_amt', 'uds_order_last90.fair_market_value_amt', 'total_orders.num_products_sum_90d_1d', 'total_orders.total_spent_sum_90d_1d', 'total_orders.total_gcr_sum_90d_1d', 'total_orders.total_fair_market_value_sum_90d_1d', 'total_orders.num_products_sum_90d_1d_shopperml']

In [76]:
df_merged_new["total_orders.num_products_sum_90d_1d_shopperml"] = df_merged["uds_order_last90.receipt_qty"].apply(sum)
df_merged_new["total_orders.total_spent_sum_90d_1d_shopperml"] = df_merged["uds_order_last90.receipt_price_amt"].apply(sum)
df_merged_new["total_orders.total_gcr_sum_90d_1d_shopperml"] = df_merged["uds_order_last90.gcr_amt"].apply(sum)
df_merged_new["total_orders.total_fair_market_value_sum_90d_1d_shopperml"] = df_merged["uds_order_last90.fair_market_value_amt"].apply(sum)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

<string>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
<string>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
<string>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
<string>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFra

In [77]:
df_merged_new.columns.tolist()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

['shopper_id', 'uds_order_last90.receipt_qty', 'uds_order_last90.receipt_price_amt', 'uds_order_last90.gcr_amt', 'uds_order_last90.fair_market_value_amt', 'total_orders.num_products_sum_90d_1d', 'total_orders.total_spent_sum_90d_1d', 'total_orders.total_gcr_sum_90d_1d', 'total_orders.total_fair_market_value_sum_90d_1d', 'total_orders.num_products_sum_90d_1d_shopperml', 'total_orders.total_spent_sum_90d_1d_shopperml', 'total_orders.total_gcr_sum_90d_1d_shopperml', 'total_orders.total_fair_market_value_sum_90d_1d_shopperml']

In [81]:
df_merged_new_v2 = df_merged_new.drop(['uds_order_last90.receipt_qty',
                                       'uds_order_last90.receipt_price_amt',
                                       'uds_order_last90.gcr_amt',
                                       'uds_order_last90.fair_market_value_amt',], 1)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

<string>:4: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only

In [82]:
df_merged_new_v2.columns.tolist()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

['shopper_id', 'total_orders.num_products_sum_90d_1d', 'total_orders.total_spent_sum_90d_1d', 'total_orders.total_gcr_sum_90d_1d', 'total_orders.total_fair_market_value_sum_90d_1d', 'total_orders.num_products_sum_90d_1d_shopperml', 'total_orders.total_spent_sum_90d_1d_shopperml', 'total_orders.total_gcr_sum_90d_1d_shopperml', 'total_orders.total_fair_market_value_sum_90d_1d_shopperml']

In [92]:
df_merged_new_v2["total_orders.num_products_sum_90d_diff"] = df_merged_new_v2["total_orders.num_products_sum_90d_1d"] - df_merged_new_v2["total_orders.num_products_sum_90d_1d_shopperml"]

df_merged_new_v2["total_orders.total_spent_sum_90d_diff"] = df_merged_new_v2["total_orders.total_spent_sum_90d_1d"] - df_merged_new_v2["total_orders.total_spent_sum_90d_1d_shopperml"]

df_merged_new_v2["total_orders.total_gcr_sum_90d_diff"] = df_merged_new_v2["total_orders.total_gcr_sum_90d_1d"] - df_merged_new_v2["total_orders.total_gcr_sum_90d_1d_shopperml"]

df_merged_new_v2["total_orders.total_fair_market_value_sum_90d_diff"] = df_merged_new_v2["total_orders.total_fair_market_value_sum_90d_1d"] - df_merged_new_v2["total_orders.total_fair_market_value_sum_90d_1d_shopperml"]


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [93]:
df_merged_new_v2.to_csv("df_merged_new_v2.csv")

import boto3
s3 = boto3.Session().resource('s3')

s3.Bucket("gd-gdmltecton-stage-feature-data").Object("wdd-model/wdd_us_labeled_20211105/df_merged_new_v2.csv").upload_file('df_merged_new_v2.csv')

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [67]:
temp = [1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 500.0, 498.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 499.0, 1.0, 1.0, 1.0, 4.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 494.0, 1.0, 495.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 992.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 500.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 500.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 496.0, 997.0, 1.0, 990.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 499.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0]

sum(temp)

An error was encountered:
Invalid status code '404' from http://tecton-notebook-emr-master.cluster.internal:8998/sessions/5 with error payload: {"msg":"Session '5' not found."}


In [71]:
dataset_location = "s3://gd-gdmltecton-stage-feature-data/wdd-model/wdd_us_labeled_20211105/full.parquet/dataset_00000_no_orders.parquet/"
wdd_df = spark.read.parquet(dataset_location)
wdd_df_new = wdd_df.toPandas()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [73]:
wdd_df_new.columns

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Index(['shopper_id', 'number_of_domains', 'c_xxx_countrycode', 'label',
       'marketing_cdl.b_xxx_china_cust',
       'marketing_cdl.b_xxx_first_order_viral',
       'marketing_cdl.b_xxx_first_pop_c3', 'marketing_cdl.b_xxx_iscompany',
       'marketing_cdl.b_xxx_isattrition_computed',
       'marketing_cdl.b_xxx_isdonotcall', 'marketing_cdl.b_xxx_isreseller',
       'marketing_cdl.d_bin_last_mm_order_days',
       'marketing_cdl.d_bin_last_bb_order_days',
       'marketing_cdl.d_bin_last_email_order_days',
       'marketing_cdl.d_bin_last_bb_contact_days',
       'marketing_cdl.d_bin_last_mm_contact_days',
       'marketing_cdl.d_bin_last_email_contact_days',
       'marketing_cdl.d_bin_last_bb_open_days',
       'marketing_cdl.d_bin_last_mm_open_days',
       'marketing_cdl.d_bin_last_email_open_days',
       'marketing_cdl.i_xxx_bb_contacts_30d',
       'marketing_cdl.i_xxx_bb_contacts_90d',
       'marketing_cdl.i_xxx_bb_contacts_1y',
       'marketing_cdl.i_xxx_bb_opens_90d', 'ma

In [75]:
wdd_df_new["bill_modified_mst_ts"] = pd.Timestamp('2022-05-21 12:00Z')
spine = pd.DataFrame()
spine["shopper_id"] = wdd_df_new["shopper_id"]
spine["bill_modified_mst_ts"] = wdd_df_new["bill_modified_mst_ts"]

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [76]:
my_workspace = tecton.get_workspace("prod")
my_fs = my_workspace.get_feature_service("wdd_service")
temp_df = my_fs.get_historical_features(spine).to_spark()
temp_df.columns

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

['shopper_id', 'bill_modified_mst_ts', 'total_orders__num_products_sum_90d_1d', 'total_orders__total_spent_sum_90d_1d', 'total_orders__total_gcr_sum_90d_1d', 'total_orders__total_fair_market_value_sum_90d_1d', 'total_orders__diff_fmv_receipt_sum_90d_1d', 'customer_orders_lifetime__non_top_20_order_count', 'customer_orders_lifetime__non_top_20_total_quantity', 'customer_orders_lifetime__non_top_20_total_spent', 'customer_orders_lifetime__domain_name_registration_order_count', 'customer_orders_lifetime__domain_name_registration_total_quantity', 'customer_orders_lifetime__domain_name_registration_total_spent', 'customer_orders_lifetime__shared_hosting_order_count', 'customer_orders_lifetime__shared_hosting_total_quantity', 'customer_orders_lifetime__shared_hosting_total_spent', 'customer_orders_lifetime__domains_by_proxy_order_count', 'customer_orders_lifetime__domains_by_proxy_total_quantity', 'customer_orders_lifetime__domains_by_proxy_total_spent', 'customer_orders_lifetime__website_bu

In [77]:
tecton_df = temp_df.toPandas()
df_merged = pd.merge(wdd_df_new, tecton_df, how='inner', on = 'shopper_id')
df_merged.columns

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Index(['shopper_id', 'number_of_domains', 'c_xxx_countrycode', 'label',
       'marketing_cdl.b_xxx_china_cust',
       'marketing_cdl.b_xxx_first_order_viral',
       'marketing_cdl.b_xxx_first_pop_c3', 'marketing_cdl.b_xxx_iscompany',
       'marketing_cdl.b_xxx_isattrition_computed',
       'marketing_cdl.b_xxx_isdonotcall',
       ...
       'customer_orders_lifetime__domain_alert_total_spent',
       'customer_orders_lifetime__virtual_hosting_order_count',
       'customer_orders_lifetime__virtual_hosting_total_quantity',
       'customer_orders_lifetime__virtual_hosting_total_spent',
       'customer_orders_lifetime__online_storage_order_count',
       'customer_orders_lifetime__online_storage_total_quantity',
       'customer_orders_lifetime__online_storage_total_spent',
       'customer_orders_lifetime__online_calendar_order_count',
       'customer_orders_lifetime__online_calendar_total_quantity',
       'customer_orders_lifetime__online_calendar_total_spent'],
      dtype='ob